In [5]:
# import nltk
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/claudiostaub/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
# Functions file for personality prediction file

import unicodedata
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import string
import re
from bs4 import BeautifulSoup

REPLACE_BY_SPACE_RE = re.compile('[/(){}\\[\]\|?@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
punctuation_ = set(string.punctuation)
stopwords_ = set(stopwords.words('english'))

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()

def filter_tokens(sent):
    return ([w for w in sent if not w in stopwords_ and not w in punctuation_])

def remove_link(sent):
    return [s for s in sent if 'http' not in s]

def wt(text):
    return [word_tokenize(sent) for sent in text.split()]

def flatten(lst):
    return [item for sublist in lst for item in sublist]

def snow_stem(text):
    snowball = SnowballStemmer('english')
    return [snowball.stem(word) for word in text]

def rm_punc(sent):
    return [w for w in sent if not w in punctuation_]

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in stopwords_)
    return text

def clean_df(df):
    """Normalize the data, change all letters to lower case, split into 
    sentences,remove website links, tokenize words into a new dataframe then:
    filter punctuation, filter stopwords, stemminize
    INPUT
    --------------
    df = Pandas DataFrame
    OUTPUT
    --------------
    word = Pandas DataFrames
    ______________
    """
    newdf = df.copy()
    newdf.posts = newdf.posts.apply(lambda x: remove_accents(x).lower())
    newdf = pd.DataFrame((newdf.type,newdf.posts.apply(lambda x: x.split('|||')))).T
    newdf.posts = newdf.posts.apply(lambda x: remove_link(x))
    newdf.posts = newdf.posts.apply(lambda x: ' '.join(x))
    newdf.posts = newdf.posts.apply(lambda x: clean_text(x))
    newdf.posts = newdf.posts.apply(lambda x: wt(x))
    newdf.posts = newdf.posts.apply(lambda x: flatten(x))
    newdf.posts = newdf.posts.apply(lambda x: snow_stem(x))
    newdf.posts = newdf.posts.apply(lambda x: ' '.join(x))
    return newdf

In [115]:
df = pd.read_json('data/data.json')

In [116]:
def fraud(string):
    if 'fraud' in string:
        return True
    else:
        return False

In [117]:
df['fraud'] = df.acct_type.apply(lambda x: fraud(x))

In [118]:
df = df[['description', 'fraud']]

In [119]:
df['description'] = df.description.apply(lambda x: BeautifulSoup(x).text)

In [120]:
df['description'] = df.description.apply(lambda x: REPLACE_BY_SPACE_RE.sub(' ', x))

In [121]:

## BAD DONT RUN ME

# df['description'] = df.description.apply(lambda x: BAD_SYMBOLS_RE.sub('', x))

In [122]:
df1 = df.copy()

In [123]:
df1.description = df1.description.apply(lambda x: remove_accents(x).lower())

In [124]:
df1 = pd.DataFrame((df1.fraud, df1.description.apply(lambda x: x.split('|||')))).T

In [125]:
df1

,fraud,description
0,True,[\n \nparty starz entertaintment & diverse int...
1,False,[join us for a quick one-night community-bui...
2,False,[teacher training on the desired results devel...
3,False,[affiliations are now due in respect of the 20...
4,False,[writers and filmmakers need to understand the...
5,False,[come join the bluegrass stallions as they tak...
6,False,[ \n \n ]
7,False,[when jekyll met hyde\nthe magnetic theatre in...
8,False,[merchants exchange productions and \nthe juli...
9,False,[when jekyll met hyde\nthe magnetic theatre in...


In [113]:
# df1.description =  df1.description.apply(lambda x: remove_link(x))

In [114]:
df1

,fraud,description
0,True,[\n \nparty starz entertaintment & diverse int...
1,False,[join us for a quick one-night community-bui...
2,False,[teacher training on the desired results devel...
3,False,[affiliations are now due in respect of the 20...
4,False,[writers and filmmakers need to understand the...
5,False,[come join the bluegrass stallions as they tak...
6,False,[ \n \n ]
7,False,[]
8,False,[merchants exchange productions and \nthe juli...
9,False,[]


In [126]:
df1.description = df1.description.apply(lambda x: ' '.join(x))

In [127]:
df1

,fraud,description
0,True,\n \nparty starz entertaintment & diverse inte...
1,False,join us for a quick one-night community-buil...
2,False,teacher training on the desired results develo...
3,False,affiliations are now due in respect of the 201...
4,False,writers and filmmakers need to understand thei...
5,False,come join the bluegrass stallions as they take...
6,False,\n \n
7,False,when jekyll met hyde\nthe magnetic theatre in ...
8,False,merchants exchange productions and \nthe julia...
9,False,when jekyll met hyde\nthe magnetic theatre in ...


In [128]:
df1.description = df1.description.apply(lambda x: clean_text(x))

In [129]:
df1

,fraud,description
0,True,party starz entertaintment diverse internation...
1,False,join us quick onenight communitybuilding trip ...
2,False,teacher training desired results developmental...
3,False,affiliations due respect 2013 calendar year im...
4,False,writers filmmakers need understand legal right...
5,False,come join bluegrass stallions take rochester r...
6,False,
7,False,jekyll met hydethe magnetic theatre magnetic f...
8,False,merchants exchange productions julia morgan ba...
9,False,jekyll met hydethe magnetic theatre magnetic f...


In [47]:
# nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/claudiostaub/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [130]:
df1.description = df1.description.apply(lambda x: wt(x))

In [131]:
df1

,fraud,description
0,True,"[[party], [starz], [entertaintment], [diverse]..."
1,False,"[[join], [us], [quick], [onenight], [community..."
2,False,"[[teacher], [training], [desired], [results], ..."
3,False,"[[affiliations], [due], [respect], [2013], [ca..."
4,False,"[[writers], [filmmakers], [need], [understand]..."
5,False,"[[come], [join], [bluegrass], [stallions], [ta..."
6,False,[]
7,False,"[[jekyll], [met], [hydethe], [magnetic], [thea..."
8,False,"[[merchants], [exchange], [productions], [juli..."
9,False,"[[jekyll], [met], [hydethe], [magnetic], [thea..."


In [132]:
df1.description = df1.description.apply(lambda x: flatten(x))

In [133]:
df1

,fraud,description
0,True,"[party, starz, entertaintment, diverse, intern..."
1,False,"[join, us, quick, onenight, communitybuilding,..."
2,False,"[teacher, training, desired, results, developm..."
3,False,"[affiliations, due, respect, 2013, calendar, y..."
4,False,"[writers, filmmakers, need, understand, legal,..."
5,False,"[come, join, bluegrass, stallions, take, roche..."
6,False,[]
7,False,"[jekyll, met, hydethe, magnetic, theatre, magn..."
8,False,"[merchants, exchange, productions, julia, morg..."
9,False,"[jekyll, met, hydethe, magnetic, theatre, magn..."


In [134]:
df1.description = df1.description.apply(lambda x: snow_stem(x))

In [135]:
df1

,fraud,description
0,True,"[parti, starz, entertaint, divers, intern, gro..."
1,False,"[join, us, quick, onenight, communitybuild, tr..."
2,False,"[teacher, train, desir, result, development, p..."
3,False,"[affili, due, respect, 2013, calendar, year, i..."
4,False,"[writer, filmmak, need, understand, legal, rig..."
5,False,"[come, join, bluegrass, stallion, take, roches..."
6,False,[]
7,False,"[jekyl, met, hydeth, magnet, theatr, magnet, f..."
8,False,"[merchant, exchang, product, julia, morgan, ba..."
9,False,"[jekyl, met, hydeth, magnet, theatr, magnet, f..."


In [56]:
# df1.description = df1.description.apply(lambda x: ' '.join(x))
# df1

In [147]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, precision_recall_fscore_support, f1_score
from sklearn.linear_model import SGDClassifier

In [136]:
vectorize = TfidfVectorizer(lowercase=True, stop_words='english',
                          max_features=5000, ngram_range=(1,3))

In [137]:
X = vectorize.fit_transform(df.description)
y = df.fraud


In [138]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [139]:
from sklearn.naive_bayes import BernoulliNB

In [140]:
nb = BernoulliNB()
nb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [150]:
y_pred_nb = nb.predict(X_test)

In [151]:
recall_score(y_test, y_pred_nb)

0.770392749244713

In [152]:
precision_score(y_test, y_pred_nb)

0.18214285714285713

In [148]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)

/Users/claudiostaub/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [149]:
y_pred_sgd = sgd.predict(X_test)

In [153]:
recall_score(y_test, y_pred_sgd)

0.10574018126888217

In [ ]:
precision_score(y_test, y_)